### Latent Semantic Analysis

#### Natural Language Processing ~ A family of techniques used to derive meaning from data

#### Document ~ A collection of words--the or "rows" of our dataset

#### Body ~ A collection of documents--our entire dataset

#### Dictionary ~ The set of all words that appear in at least one document in our body 

#### Topic ~ A collection of words that co-occur 

#### Latent ~ Features that are "hidden" in the data

Unsupervised

aim - to create representations of the text data in terms of these topics or latent features

2 steps
1) Document Term Matrix
2) Singular Value Decomposition ~ performed on Document Term Matrix

You can represent documents as vectors (i.e. you can plot data points on a chart with text data if you turn it into words)

Output - topic encoded data

source: https://www.youtube.com/watch?v=hB51kkus-Rc


Rows = The entire sentence (Document). 
Coumns = 1 if document contains the column word (contained in the set of all words--Dictionary)

Then perform SVD (same idea as PCA)

The latent features represent topics or collections of words that co-occur

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer
test = "test string is bing stratus is is the is the "
test = [test]
bag_of_words =  vectorizer().fit_transform(test)
bag_of_words.todense()

matrix([[1, 4, 1, 1, 1, 2]], dtype=int64)

Documents ~ Players
 ~ scouting report
 
 
use truncated SVD in sci-kit learn